In [ ]:
#!/usr/bin/env python
# coding: utf-8

# -*- encoding:utf-8 -*-
"""
  This script provides an k-BERT exmaple for classification.
"""
import os
import collections
import codecs
import sys
import json
import random
from typing import Dict, List
from multiprocessing import Process, Pool

import argparse
import numpy as np
import torch
import torch.nn as nn
from transformers import (BertConfig, BertTokenizer, 
                          BertModel, BertPreTrainedModel,
                          DistilBertConfig, DistilBertTokenizer)
from modeling_distilbert import (DistilBertModel, 
                                 DistilBertPreTrainedModel)

from uer.optimizers import BertAdam
from brain import KnowledgeGraph
from utils import set_seed, load_hyperparam, save_model
from constants import * 

# student task model: DistilModel
class DistilbertClassifierModel(DistilBertPreTrainedModel):
    """
    DistilbertClassifierModel

    Distil model class based on huggingface class but with
    initialization in it. Model will take vocabulary
    layers from specified teacher model
    """

    def __init__(
        self, config, args, origin_model,
        layers: List[int] = None
    ):
        """
        Args:
            layers: layers indexes to initialize
            extract: bool flag, if you want to initialize your model with
                layers of the teacher model then set this to true
        """
        super().__init__(config)
   
        self.layers = [0, 2, 4, 7, 9, 11]
        self.labels_num = args.labels_num
        self.origin = origin_model #载入一个distilBert模型(无task layer 无state_dict)                                         
        self.pre_classifier = nn.Linear(args.hidden_size, args.hidden_size)
        self.classifier = nn.Linear(args.hidden_size, args.labels_num)
        
        self.softmax = nn.LogSoftmax(dim=-1)
        self.dropout = nn.Dropout(0.2)
        self.criterion = nn.NLLLoss()

        self.init_weights()
        if layers is not None:
            self.layers = layers

    def forward(
        self, 
        input_ids=None,
        attention_mask=None,
        position_ids=None, 
        token_type_ids=None, 
        head_mask=None,
        inputs_embeds=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=True,
        label_ids = None
    ):
        r"""
        labels (:obj:`torch.LongTensor` of shape :obj:`(batch_size,)`, `optional`):
            Labels for computing the sequence classification/regression loss.
            Indices should be in :obj:`[0, ..., config.num_labels - 1]`.
            If :obj:`config.num_labels == 1` a regression loss is computed (Mean-Square loss),
            If :obj:`config.num_labels > 1` a classification loss is computed (Cross-Entropy).
        """
        distilbert_output = self.origin(
            input_ids=input_ids,
            attention_mask=attention_mask,
            position_ids=position_ids, 
            token_type_ids=token_type_ids, 
            head_mask=None,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict
        )
        hidden_state = distilbert_output[0]  # (bs, seq_len, dim)
        pooled_output = hidden_state[:, 0]  # (bs, dim)
        pooled_output = self.pre_classifier(pooled_output)  # (bs, dim)
        pooled_output = nn.ReLU()(pooled_output)  # (bs, dim)
        pooled_output = self.dropout(pooled_output)  # (bs, dim)
        logits = self.classifier(pooled_output)  # (bs, dim)
        loss = self.criterion(self.softmax(logits.view(-1, self.labels_num)), label_ids.view(-1))

        return loss, logits

    @staticmethod
    def _extract(
        teacher_model,
        layers: List[int]
    ) -> Dict[str, torch.Tensor]:
        """
        Extracts state dict from teacher model

        Args:
            teacher_model: model to extract
            layers: layers indexes to initialize
            prefix_teacher: name of the teacher model
            prefix_student: name of the student model
        """
        state_dict = teacher_model.state_dict()
        compressed_sd = {}

        # extract embeddings
        compressed_sd["embeddings.position_ids"] = state_dict["embeddings.position_ids"]
        for w in ["word_embeddings", "position_embeddings", "token_type_embeddings"]:
            compressed_sd[
                f"embeddings.{w}.weight"
            ] = state_dict[f"embeddings.{w}.weight"]
        for w in ["weight", "bias"]:
            compressed_sd[
                f"embeddings.LayerNorm.{w}"
            ] = state_dict[f"embeddings.LayerNorm.{w}"]
        # extract encoder
        std_idx = 0
        for teacher_idx in layers:
            for w in ["weight", "bias"]:
                compressed_sd[
                    f"transformer.layer.{std_idx}.attention.q_lin.{w}"  # noqa: E501
                ] = state_dict[
                    f"encoder.layer.{teacher_idx}.attention.self.query.{w}"  # noqa: E501
                ]
                compressed_sd[
                    f"transformer.layer.{std_idx}.attention.k_lin.{w}"  # noqa: E501
                ] = state_dict[
                    f"encoder.layer.{teacher_idx}.attention.self.key.{w}"  # noqa: E501
                ]
                compressed_sd[
                    f"transformer.layer.{std_idx}.attention.v_lin.{w}"  # noqa: E501
                ] = state_dict[
                    f"encoder.layer.{teacher_idx}.attention.self.value.{w}"  # noqa: E501
                ]

                compressed_sd[
                    f"transformer.layer.{std_idx}.attention.out_lin.{w}"  # noqa: E501
                ] = state_dict[
                    f"encoder.layer.{teacher_idx}.attention.output.dense.{w}"  # noqa: E501
                ]
                compressed_sd[
                    f"transformer.layer.{std_idx}.sa_layer_norm.{w}"  # noqa: E501
                ] = state_dict[
                    f"encoder.layer.{teacher_idx}.attention.output.LayerNorm.{w}"  # noqa: E501
                ]

                compressed_sd[
                    f"transformer.layer.{std_idx}.ffn.lin1.{w}"  # noqa: E501
                ] = state_dict[
                    f"encoder.layer.{teacher_idx}.intermediate.dense.{w}"  # noqa: E501
                ]
                compressed_sd[
                    f"transformer.layer.{std_idx}.ffn.lin2.{w}"  # noqa: E501
                ] = state_dict[
                    f"encoder.layer.{teacher_idx}.output.dense.{w}"  # noqa: E501
                ]
                compressed_sd[
                    f"transformer.layer.{std_idx}.output_layer_norm.{w}"  # noqa: E501
                ] = state_dict[
                    f"encoder.layer.{teacher_idx}.output.LayerNorm.{w}"  # noqa: E501
                ]

            std_idx += 1
        return compressed_sd

def add_knowledge_worker(params):
    p_id, sentences, columns, kg, bert_token, max_length = params # modified
    sentences_num = len(sentences)
    dataset = []
    for line_id, line in enumerate(sentences):
        if line_id % 10000 == 0:
            print("Progress of process {}: {}/{}".format(p_id, line_id, sentences_num))
            sys.stdout.flush()
        line = line.strip().split('\t')
        try:
            label = int(line[columns["label"]])
            if len(line) == 2:
                text = CLS_TOKEN + line[columns["text_a"]]
   
                tokens, pos, vm, _ = kg.add_knowledge_with_vm([text], add_pad=True, max_length=max_length) # modified
                tokens = tokens[0]
                pos = pos[0]
                vm = vm[0]
                token_ids = bert_token.convert_tokens_to_ids(tokens)

                seg = [0 for t in tokens]
                
                dataset.append((token_ids, label, seg, pos, vm))
            
            elif len(line)==3:                  
                text_a, text_b = line[columns["text_a"]], line[columns["text_b"]]
                text = CLS_TOKEN + text_a + SEP_TOKEN + text_b + SEP_TOKEN

                tokens, pos, vm, _ = kg.add_knowledge_with_vm([text], add_pad=True, max_length=max_length) # modified
                tokens = tokens[0]
                pos = pos[0]
                vm = vm[0]
                token_ids = bert_token.convert_tokens_to_ids(tokens)

                seg = []
                seg_tag = 0
                for t in tokens:
                    seg.append(seg_tag)
                    if t == SEP_TOKEN:
                        seg_tag += 1
                
                dataset.append((token_ids, label, seg, pos, vm))
            
            elif len(line) == 4:  # for dbqa
                qid=int(line[columns["qid"]])
                text_a, text_b = line[columns["text_a"]], line[columns["text_b"]]
                text = CLS_TOKEN + text_a + SEP_TOKEN + text_b + SEP_TOKEN

                tokens, pos, vm, _ = kg.add_knowledge_with_vm([text], add_pad=True, max_length=max_length) # modified
                tokens = tokens[0]
                pos = pos[0]
                vm = vm[0]
                token_ids = bert_token.convert_tokens_to_ids(tokens)

                seg = []
                seg_tag = 0
                for t in tokens:
                    seg.append(seg_tag)
                    if t == SEP_TOKEN:
                        seg_tag += 1
                
                dataset.append((token_ids, label, seg, pos, vm, qid))
            else:
                pass
            
        except:
            print("Error line: ", line)
    return dataset

In [ ]:
args = {    
    "teacher_model_path": "./models/bert_origin/",
    "teacher_config_path": "./models/bert_origin/config.json",
    "model_path": "./models/distilbert-chinese/",
    "config_path": "./models/distilbert-chinese/config.json",
    
#     "train_path": "/input/datasets_K-BERT/nlpcc-dbqa/train.tsv",
#     "dev_path":  "/input/datasets_K-BERT/nlpcc-dbqa/dev.tsv",
#     "test_path":  "/input/datasets_K-BERT/nlpcc-dbqa/test.tsv",
#     "output_model_path": "./outputs/kbert_nlpcc-dbqa_CnDbpedia.bin",
    "train_path": "/input/datasets_K-BERT/weibo/train.tsv",
    "dev_path":  "/input/datasets_K-BERT/weibo/dev.tsv",
    "test_path":  "/input/datasets_K-BERT/weibo/test.tsv",
    "output_model_path": "./outputs/distilbert_book_review_CnDbpedia.bin",
    
    "kg_name": "CnDbpedia",

    "batch_size": 128, # 32, 64, 128
    "seq_length": 256,
    "learning_rate":2e-5 ,
    "warmup": 0.1,
    "dropout": 0.5,
    "epochs_num": 10, # 5, 10, 20
    "report_steps": 100,
    "seed": 7,
    "mean_reciprocal_rank": False, # True for DBQA dataset
    "workers_num": 1, # number of process for loading dataset，取决于cpu数量和线程数量
    "no_vm": False, # Disable the visible_matrix
}

class Args(dict):  #字典转对象，递归版,既可以作为对象、也可以作为属性
    __setattr__ = dict.__setitem__
    __getattr__ = dict.__getitem__
args = Args(args)

# Load the hyperparameters from the config file.
args = load_hyperparam(args)

set_seed(args.seed)

# Count the number of labels.
labels_set = set()
columns = {}
with open(args.train_path, mode="r", encoding="utf-8") as f:
    for line_id, line in enumerate(f):
        try:
            line = line.strip().split("\t")
            if line_id == 0:
                for i, column_name in enumerate(line):
                    columns[column_name] = i
                continue
            label = int(line[columns["label"]])
            labels_set.add(label)
        except:
            pass
args.labels_num = len(labels_set) 

# Build knowledge graph.
if args.kg_name == 'none':
    spo_files = []
else:
    spo_files = [args.kg_name]
kg = KnowledgeGraph(spo_files=spo_files, predicate=True)

In [ ]:
## teacher origin model
bert_config = BertConfig.from_pretrained(args.teacher_model_path)
bert_token = BertTokenizer.from_pretrained(args.teacher_model_path)
bert_model = BertModel(config=bert_config)
bert_model.config.max_position_embeddings = args.seq_length #句子最大长度256
bert_model.load_state_dict(torch.load(args.teacher_model_path+'pytorch_model.bin'), strict=False)

## student origin model(model without task layer)
distilbert_config = DistilBertConfig.from_pretrained(args.model_path)
distilbert_token = bert_token #这俩在transformers库里其实一个样
distilbert_model = DistilBertModel(config=distilbert_config)
distilbert_model.config.max_position_embeddings = args.seq_length #句子最大长度256

## student task model
model = DistilbertClassifierModel(distilbert_config, args, distilbert_model)
# model.load_state_dict(torch.load(args.model_path+'pytorch_model.bin'), strict=False)
model.load_state_dict(torch.load(args.output_model_path), strict=False)

# extract teacher model to model.origin(model without task layer), just run once
# distil_sd = model._extract(bert_model, model.layers)
# model.origin.load_state_dict(distil_sd, strict=False)
# save_model(model.origin, args.output_model_path)

In [ ]:
# for i in distil_sd.keys():
#     print(i)
# for i in model.state_dict().keys():
#     print(i)
# [print(i) for i in bert_model.state_dict().keys()]
# [print(i) for i in distilbert_model.state_dict().keys()]

In [ ]:
# For simplicity, we use DataParallel wrapper to use multiple GPUs.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.device_count() > 1:
    print("{} GPUs are available. Let's use them.".format(torch.cuda.device_count()))
    model = nn.DataParallel(model)

model = model.to(device)

In [ ]:
# Datset loader.
def batch_loader(batch_size, input_ids, label_ids, seg_ids, pos_ids, vms):
    instances_num = input_ids.size()[0]
    for i in range(instances_num // batch_size):
        input_ids_batch = input_ids[i*batch_size: (i+1)*batch_size, :]
        label_ids_batch = label_ids[i*batch_size: (i+1)*batch_size]
        seg_ids_batch = seg_ids[i*batch_size: (i+1)*batch_size, :]
        pos_ids_batch = pos_ids[i*batch_size: (i+1)*batch_size, :]
        vms_batch = vms[i*batch_size: (i+1)*batch_size]
        yield input_ids_batch, label_ids_batch, seg_ids_batch, pos_ids_batch, vms_batch

    if instances_num > instances_num // batch_size * batch_size:
        input_ids_batch = input_ids[instances_num//batch_size*batch_size:, :]
        label_ids_batch = label_ids[instances_num//batch_size*batch_size:]
        seg_ids_batch = seg_ids[instances_num//batch_size*batch_size:, :]
        pos_ids_batch = pos_ids[instances_num//batch_size*batch_size:, :]
        vms_batch = vms[instances_num//batch_size*batch_size:]
        yield input_ids_batch, label_ids_batch, seg_ids_batch, pos_ids_batch, vms_batch

def read_dataset(path, workers_num=1):

    print("Loading sentences from {}".format(path))
    sentences = []
    with open(path, mode='r', encoding="utf-8") as f:
        for line_id, line in enumerate(f):
            if line_id == 0:
                continue
            sentences.append(line)
    sentence_num = len(sentences)

    print("There are {} sentence in total. We use {} processes to inject knowledge into sentences.".format(sentence_num, workers_num))
    if workers_num > 1:
        params = []
        sentence_per_block = int(sentence_num / workers_num) + 1
        for i in range(workers_num):
            params.append((i, sentences[i*sentence_per_block: (i+1)*sentence_per_block], columns, kg, bert_token, args.seq_length)) # modified
        pool = Pool(workers_num)
        res = pool.map(add_knowledge_worker, params)
        pool.close()
        pool.join()
        dataset = [sample for block in res for sample in block]
    else:
        params = (0, sentences, columns, kg, bert_token, args.seq_length) # modified
        dataset = add_knowledge_worker(params)

    return dataset

# Evaluation function.
def evaluate(args, is_test, metrics='Acc'):
    if is_test:
        dataset = read_dataset(args.test_path, workers_num=args.workers_num)
    else:
        dataset = read_dataset(args.dev_path, workers_num=args.workers_num)

    input_ids = torch.LongTensor([sample[0] for sample in dataset])
    label_ids = torch.LongTensor([sample[1] for sample in dataset])
    seg_ids = torch.LongTensor([sample[2] for sample in dataset])
    pos_ids = torch.LongTensor([example[3] for example in dataset])
    vms = [example[4] for example in dataset] # list of 2-dim tensor

    batch_size = args.batch_size
    instances_num = input_ids.size()[0]
    if is_test:
        print("The number of evaluation instances: ", instances_num)

    correct = 0
    # Confusion matrix.
    confusion = torch.zeros(args.labels_num, args.labels_num, dtype=torch.long)

    model.eval()

    if not args.mean_reciprocal_rank:
        for i, (input_ids_batch, label_ids_batch,  seg_ids_batch, pos_ids_batch, vms_batch) in enumerate(batch_loader(batch_size, input_ids, label_ids, seg_ids, pos_ids, vms)):

            # vms_batch = vms_batch.long()
            vms_batch = torch.LongTensor(vms_batch)

            input_ids_batch = input_ids_batch.to(device)
            label_ids_batch = label_ids_batch.to(device)
            seg_ids_batch = seg_ids_batch.to(device)
            pos_ids_batch = pos_ids_batch.to(device)
            vms_batch = vms_batch.to(device)

            with torch.no_grad():
                try:
                    loss, logits = model(
                        input_ids=input_ids_batch, 
                        attention_mask=vms_batch, 
                        position_ids=pos_ids_batch,
                        token_type_ids=seg_ids_batch, 
                        label_ids=label_ids_batch
                     )
                except:
                    print(input_ids_batch)
                    print(input_ids_batch.size())
                    print(vms_batch)
                    print(vms_batch.size())

            logits = nn.Softmax(dim=1)(logits)
            pred = torch.argmax(logits, dim=1)
            gold = label_ids_batch
            for j in range(pred.size()[0]):
                confusion[pred[j], gold[j]] += 1
            correct += torch.sum(pred == gold).item()

        if is_test:
            print("Confusion matrix:")
            print(confusion)
            print("Report precision, recall, and f1:")

        for i in range(confusion.size()[0]):
            p = confusion[i,i].item()/confusion[i,:].sum().item()
            r = confusion[i,i].item()/confusion[:,i].sum().item()
            f1 = 2*p*r / (p+r)
            if i == 1:
                label_1_f1 = f1
            print("Label {}: {:.3f}, {:.3f}, {:.3f}".format(i,p,r,f1))
        print("Acc. (Correct/Total): {:.4f} ({}/{}) ".format(correct/len(dataset), correct, len(dataset)))
        if metrics == 'Acc':
            return correct/len(dataset)
        elif metrics == 'f1':
            return label_1_f1
        else:
            return correct/len(dataset)
    else:
        for i, (input_ids_batch, label_ids_batch,  seg_ids_batch, pos_ids_batch, vms_batch) in enumerate(batch_loader(batch_size, input_ids, label_ids, seg_ids, pos_ids, vms)):

            vms_batch = torch.LongTensor(vms_batch)

            input_ids_batch = input_ids_batch.to(device)
            label_ids_batch = label_ids_batch.to(device)
            seg_ids_batch = seg_ids_batch.to(device)
            pos_ids_batch = pos_ids_batch.to(device)
            vms_batch = vms_batch.to(device)

            with torch.no_grad():
                loss, logits = model(
                    input_ids=input_ids_batch, 
                    attention_mask=vms_batch, 
                    position_ids=pos_ids_batch,
                    token_type_ids=seg_ids_batch, 
                    label_ids=label_ids_batch
                 )
            logits = nn.Softmax(dim=1)(logits)
            if i == 0:
                logits_all=logits
            if i >= 1:
                logits_all=torch.cat((logits_all,logits),0)

        order = -1
        gold = []
        for i in range(len(dataset)):
            qid = dataset[i][-1]
            label = dataset[i][1]
            if qid == order:
                j += 1
                if label == 1:
                    gold.append((qid,j))
            else:
                order = qid
                j = 0
                if label == 1:
                    gold.append((qid,j))

        label_order = []
        order = -1
        for i in range(len(gold)):
            if gold[i][0] == order:
                templist.append(gold[i][1])
            elif gold[i][0] != order:
                order=gold[i][0]
                if i > 0:
                    label_order.append(templist)
                templist = []
                templist.append(gold[i][1])
        label_order.append(templist)

        order = -1
        score_list = []
        for i in range(len(logits_all)):
            score = float(logits_all[i][1])
            qid=int(dataset[i][-1])
            if qid == order:
                templist.append(score)
            else:
                order = qid
                if i > 0:
                    score_list.append(templist)
                templist = []
                templist.append(score)
        score_list.append(templist)

        rank = []
        pred = []
        print(len(score_list))
        print(len(label_order))
        for i in range(len(score_list)):
            if len(label_order[i])==1:
                if label_order[i][0] < len(score_list[i]):
                    true_score = score_list[i][label_order[i][0]]
                    score_list[i].sort(reverse=True)
                    for j in range(len(score_list[i])):
                        if score_list[i][j] == true_score:
                            rank.append(1 / (j + 1))
                else:
                    rank.append(0)

            else:
                true_rank = len(score_list[i])
                for k in range(len(label_order[i])):
                    if label_order[i][k] < len(score_list[i]):
                        true_score = score_list[i][label_order[i][k]]
                        temp = sorted(score_list[i],reverse=True)
                        for j in range(len(temp)):
                            if temp[j] == true_score:
                                if j < true_rank:
                                    true_rank = j
                if true_rank < len(score_list[i]):
                    rank.append(1 / (true_rank + 1))
                else:
                    rank.append(0)
        MRR = sum(rank) / len(rank)
        print("MRR", MRR)
        return MRR


In [ ]:
# Training phase.
print("Start training.")
trainset = read_dataset(args.train_path, workers_num=args.workers_num)
print("Shuffling dataset")
random.shuffle(trainset)
instances_num = len(trainset)
batch_size = args.batch_size

print("Trans data to tensor.")
print("input_ids")
input_ids = torch.LongTensor([example[0] for example in trainset])
print("label_ids")
label_ids = torch.LongTensor([example[1] for example in trainset])
print("seg_ids")
seg_ids = torch.LongTensor([example[2] for example in trainset])
print("pos_ids")
pos_ids = torch.LongTensor([example[3] for example in trainset])
print("vms")
vms = [example[4] for example in trainset]

train_steps = int(instances_num * args.epochs_num / batch_size) + 1

print("Batch size: ", batch_size)
print("The number of training instances:", instances_num)

param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
            {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay_rate': 0.01},
            {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay_rate': 0.0}
]
optimizer = BertAdam(optimizer_grouped_parameters, lr=args.learning_rate, warmup=args.warmup, t_total=train_steps)

total_loss = 0.
result = 0.0
best_result = 0.0
# best_result = 0.0


In [ ]:
# Training phase 2
for epoch in range(1, args.epochs_num+1):
    model.train()
    for i, (input_ids_batch, label_ids_batch, seg_ids_batch, pos_ids_batch, vms_batch) in enumerate(batch_loader(batch_size, input_ids, label_ids, seg_ids, pos_ids, vms)):
        model.zero_grad()

        vms_batch = torch.LongTensor(vms_batch)

        input_ids_batch = input_ids_batch.to(device)
        label_ids_batch = label_ids_batch.to(device)
        seg_ids_batch = seg_ids_batch.to(device)
        pos_ids_batch = pos_ids_batch.to(device)
        vms_batch = vms_batch.to(device)

        loss, logits = model(
            input_ids=input_ids_batch, 
            attention_mask=vms_batch, 
            position_ids=pos_ids_batch,
            token_type_ids=seg_ids_batch, 
            label_ids=label_ids_batch
         )
        if torch.cuda.device_count() > 1:
            loss = torch.mean(loss)
        total_loss += loss.item()
        if (i + 1) % args.report_steps == 0:
            print("Epoch id: {}, Training steps: {}, Avg loss: {:.3f}".format(epoch, i+1, total_loss / args.report_steps))
            sys.stdout.flush()
            total_loss = 0.
        loss.backward()
        optimizer.step()

    print("Start evaluation on dev dataset.")
    result = evaluate(args, False)
    if result > best_result:
        best_result = result
        save_model(model, args.output_model_path)
    else:
        continue

    print("Start evaluation on test dataset.")
    evaluate(args, True)


In [ ]:
# Evaluation phase.
print("Final evaluation on the test dataset.")

if torch.cuda.device_count() > 1:
    model.module.load_state_dict(torch.load(args.output_model_path))
else:
    model.load_state_dict(torch.load(args.output_model_path))
evaluate(args, True)